# 📊 Lezione 18 — Metriche di Valutazione per la Regressione

## 🎯 Obiettivi della Lezione

In questa lezione imparerai:

1. **Perché non basta R²** — limiti e interpretazione
2. **MAE, MSE, RMSE** — errori assoluti e quadratici
3. **MAPE** — errore percentuale (interpretabilità business)
4. **Confronto tra metriche** — quando usare quale
5. **Residui** — analisi diagnostica del modello

---

## 🧠 Il Problema: Come Valuto un Modello di Regressione?

### Classificazione vs Regressione

| Classificazione | Regressione |
|-----------------|-------------|
| Predice classi discrete | Predice valori continui |
| Accuracy, Precision, Recall | MAE, MSE, R² |
| "Giusto o sbagliato" | "Quanto sbaglio?" |

### Domanda Chiave

> In regressione non c'è "giusto/sbagliato". C'è solo **quanto sei lontano** dal valore vero.

**Esempio:** Predici che una casa costa €200.000, il prezzo reale è €210.000.
- Non è "sbagliato" in senso assoluto
- L'errore è €10.000 (o 5%)

---

## 1️⃣ MAE — Mean Absolute Error

### Formula

$$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$

### Interpretazione

| Aspetto | Descrizione |
|---------|-------------|
| **Cosa misura** | Media degli errori assoluti |
| **Unità** | Stessa unità di y (€, kg, anni...) |
| **Range** | [0, ∞) — 0 = perfetto |
| **Sensibilità outlier** | Bassa (robusto) |

### Pro e Contro

| ✅ Pro | ❌ Contro |
|--------|----------|
| Facile da interpretare | Non penalizza grandi errori |
| Robusto agli outlier | Non differenziabile in 0 |
| Stessa scala di y | Può sottovalutare errori gravi |

### Esempio Intuitivo

```
Predizioni: [100, 200, 150]
Valori veri: [110, 180, 160]
Errori: |110-100|=10, |180-200|=20, |160-150|=10
MAE = (10 + 20 + 10) / 3 = 13.33
```
→ "In media, sbaglio di €13.33"

---

## 2️⃣ MSE e RMSE — Mean Squared Error

### Formule

$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

$$RMSE = \sqrt{MSE}$$

### Interpretazione

| Metrica | Unità | Sensibilità Outlier |
|---------|-------|---------------------|
| **MSE** | y² (es. €²) | ALTA |
| **RMSE** | y (es. €) | ALTA |

### Perché RMSE invece di MSE?

- **MSE** è in unità quadrate → difficile da interpretare
- **RMSE** riporta alla scala originale → confrontabile con MAE

### Esempio: Effetto Outlier

```
Scenario 1: errori = [10, 10, 10]
    MAE = 10, MSE = 100, RMSE = 10

Scenario 2: errori = [1, 1, 28]  (un outlier!)
    MAE = 10, MSE = 262, RMSE = 16.2
```
→ MSE/RMSE **penalizzano molto** i grandi errori!

In [ ]:
# ============================================
# DEMO: MAE vs MSE vs RMSE
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Simuliamo previsioni di prezzi immobiliari
np.random.seed(42)

y_true = np.array([200, 250, 180, 320, 275, 190, 400, 350, 220, 280])  # Prezzi reali (k€)
y_pred = np.array([210, 240, 175, 310, 290, 200, 380, 340, 230, 270])  # Predizioni

# Calcolo metriche
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)

print("=" * 60)
print("📊 CONFRONTO METRICHE DI ERRORE")
print("=" * 60)

print(f"\n📌 Dataset: Prezzi immobiliari (k€)")
print(f"   - Campioni: {len(y_true)}")
print(f"   - Prezzo medio reale: {y_true.mean():.1f}k€")

print(f"\n📈 Metriche:")
print(f"   MAE  = {mae:.2f}k€  (errore medio assoluto)")
print(f"   MSE  = {mse:.2f}k€² (errore quadratico medio)")
print(f"   RMSE = {rmse:.2f}k€  (radice di MSE)")

# Visualizzazione errori
errori = y_true - y_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Predizioni vs Valori Reali
ax1 = axes[0]
ax1.scatter(y_true, y_pred, s=100, alpha=0.7, c='steelblue', edgecolors='black')
ax1.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 
         'r--', linewidth=2, label='Predizione perfetta')
ax1.set_xlabel('Prezzo Reale (k€)', fontsize=12)
ax1.set_ylabel('Prezzo Predetto (k€)', fontsize=12)
ax1.set_title('🎯 Predizioni vs Valori Reali', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Distribuzione errori
ax2 = axes[1]
bars = ax2.bar(range(len(errori)), errori, color=['green' if e >= 0 else 'red' for e in errori],
               edgecolor='black', alpha=0.7)
ax2.axhline(y=0, color='black', linewidth=1)
ax2.axhline(y=mae, color='blue', linestyle='--', linewidth=2, label=f'MAE = {mae:.1f}')
ax2.axhline(y=-mae, color='blue', linestyle='--', linewidth=2)
ax2.set_xlabel('Campione', fontsize=12)
ax2.set_ylabel('Errore (y_true - y_pred)', fontsize=12)
ax2.set_title('📊 Distribuzione Errori', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Interpretazione:")
print(f"   - In media sbagliamo di {mae:.1f}k€ (MAE)")
print(f"   - RMSE ({rmse:.1f}k€) > MAE → ci sono errori più grandi della media")

---

## 3️⃣ R² — Coefficiente di Determinazione

### Formula

$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum(y_i - \hat{y}_i)^2}{\sum(y_i - \bar{y})^2}$$

### Interpretazione

| Valore R² | Significato |
|-----------|-------------|
| **1.0** | Predizione perfetta |
| **0.8-1.0** | Ottimo |
| **0.6-0.8** | Buono |
| **0.4-0.6** | Moderato |
| **< 0.4** | Scarso |
| **< 0** | Peggio della media! |

### Cosa Significa R²?

> **R² = 0.75** significa: "Il modello spiega il 75% della varianza di y"

**Attenzione:** R² può essere ingannevole!
- Cresce sempre aggiungendo feature (usa **Adjusted R²**)
- Non indica se il modello è corretto, solo quanto "spiega"
- R² alto ≠ buone predizioni (può essere overfitting)

### Adjusted R²

$$R^2_{adj} = 1 - (1 - R^2) \cdot \frac{n-1}{n-p-1}$$

Dove p = numero di feature. Penalizza l'aggiunta di feature inutili.

In [ ]:
# ============================================
# DEMO: R² e Adjusted R²
# ============================================

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

# Dataset sintetico
X, y = make_regression(n_samples=100, n_features=5, n_informative=3, 
                       noise=10, random_state=42)

# Modello
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# R²
r2 = r2_score(y, y_pred)

# Adjusted R²
n = len(y)
p = X.shape[1]
r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print("=" * 60)
print("📊 R² e ADJUSTED R²")
print("=" * 60)

print(f"\n📌 Dataset:")
print(f"   - Campioni: {n}")
print(f"   - Feature: {p}")
print(f"   - Feature informative reali: 3")

print(f"\n📈 Metriche:")
print(f"   R²     = {r2:.4f}")
print(f"   R² adj = {r2_adj:.4f}")

print(f"\n💡 Interpretazione:")
print(f"   - Il modello spiega {r2*100:.1f}% della varianza")
print(f"   - Adjusted R² penalizza le 2 feature non informative")

# Visualizzazione: cosa significa R²
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Predizioni vs Reali
ax1 = axes[0]
ax1.scatter(y, y_pred, alpha=0.6, s=50, c='steelblue', edgecolors='black')
ax1.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2, 
         label='Predizione perfetta')
ax1.set_xlabel('Valore Reale', fontsize=12)
ax1.set_ylabel('Valore Predetto', fontsize=12)
ax1.set_title(f'🎯 R² = {r2:.3f}', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# R² con diversi livelli di rumore
noise_levels = [5, 20, 50, 100, 200]
r2_scores = []

for noise in noise_levels:
    X_temp, y_temp = make_regression(n_samples=100, n_features=3, n_informative=3,
                                      noise=noise, random_state=42)
    model_temp = LinearRegression()
    model_temp.fit(X_temp, y_temp)
    r2_scores.append(r2_score(y_temp, model_temp.predict(X_temp)))

ax2 = axes[1]
ax2.bar(range(len(noise_levels)), r2_scores, color='steelblue', edgecolor='black')
ax2.set_xticks(range(len(noise_levels)))
ax2.set_xticklabels([f'σ={n}' for n in noise_levels])
ax2.set_xlabel('Livello di Rumore', fontsize=12)
ax2.set_ylabel('R²', fontsize=12)
ax2.set_title('📉 R² diminuisce con il rumore', fontsize=14)
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3, axis='y')

for i, r in enumerate(r2_scores):
    ax2.text(i, r + 0.02, f'{r:.2f}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

---

## 4️⃣ MAPE — Mean Absolute Percentage Error

### Formula

$$MAPE = \frac{100\%}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right|$$

### Interpretazione

| Aspetto | Descrizione |
|---------|-------------|
| **Cosa misura** | Errore percentuale medio |
| **Unità** | Percentuale (%) |
| **Range** | [0%, ∞) |
| **Vantaggio** | Facilissimo da comunicare! |

### Pro e Contro

| ✅ Pro | ❌ Contro |
|--------|----------|
| Intuitivo per il business | Non definito se y=0 |
| Scala-indipendente | Asimmetrico (sovrastima errori su valori piccoli) |
| Confrontabile tra dataset | Non usare se y può essere 0 o negativo |

### Esempio Business

```
MAPE = 8%
```
→ "In media, le nostre previsioni sbagliano dell'8%"

Questo lo capisce chiunque!

In [ ]:
# ============================================
# DEMO: MAPE e confronto completo metriche
# ============================================

from sklearn.metrics import mean_absolute_percentage_error

# Usiamo il dataset prezzi di prima
print("=" * 60)
print("📊 MAPE — ERRORE PERCENTUALE")
print("=" * 60)

# MAPE
mape = mean_absolute_percentage_error(y_true, y_pred) * 100  # in percentuale

print(f"\n📌 Dataset: Prezzi immobiliari")
print(f"   Prezzo medio: {y_true.mean():.0f}k€")

print(f"\n📈 Metriche a confronto:")
print(f"   MAE  = {mae:.2f}k€")
print(f"   RMSE = {rmse:.2f}k€")
print(f"   MAPE = {mape:.2f}%")
print(f"   R²   = {r2_score(y_true, y_pred):.4f}")

# Visualizzazione confronto
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Errore assoluto vs percentuale
ax1 = axes[0]
errori_abs = np.abs(y_true - y_pred)
errori_perc = np.abs((y_true - y_pred) / y_true) * 100

x_pos = np.arange(len(y_true))
width = 0.35

bars1 = ax1.bar(x_pos - width/2, errori_abs, width, label='Errore Assoluto (k€)', 
                color='steelblue', edgecolor='black')
ax1_twin = ax1.twinx()
bars2 = ax1_twin.bar(x_pos + width/2, errori_perc, width, label='Errore % ', 
                     color='coral', edgecolor='black')

ax1.set_xlabel('Campione', fontsize=12)
ax1.set_ylabel('Errore Assoluto (k€)', color='steelblue', fontsize=12)
ax1_twin.set_ylabel('Errore Percentuale (%)', color='coral', fontsize=12)
ax1.set_title('📊 Errore Assoluto vs Percentuale', fontsize=14)

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twin.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Riepilogo metriche
ax2 = axes[1]
metriche = ['MAE\n(k€)', 'RMSE\n(k€)', 'MAPE\n(%)', 'R²']
valori = [mae, rmse, mape, r2_score(y_true, y_pred) * 100]  # R² scalato per visualizzazione
colori = ['#3498db', '#e74c3c', '#27ae60', '#9b59b6']

bars = ax2.bar(metriche, valori, color=colori, edgecolor='black')
ax2.set_ylabel('Valore', fontsize=12)
ax2.set_title('📊 Riepilogo Metriche', fontsize=14)

for bar, val in zip(bars, valori):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{val:.2f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Per il business:")
print(f"   'Le nostre stime dei prezzi sbagliano in media del {mape:.1f}%'")

---

## 5️⃣ Analisi dei Residui

### Cosa Sono i Residui?

$$residuo_i = y_i - \hat{y}_i$$

I residui sono gli **errori** del modello. Analizzarli ci dice se il modello è appropriato.

### Residui Ideali

Un buon modello ha residui che sono:

| Proprietà | Cosa Significa | Come Verificare |
|-----------|----------------|-----------------|
| **Media zero** | Nessun bias sistematico | mean(residui) ≈ 0 |
| **Omoscedastici** | Varianza costante | Scatter plot uniforme |
| **Normali** | Distribuzione gaussiana | Q-Q plot, istogramma |
| **Indipendenti** | No pattern | No trend nel tempo |

### Segnali di Problemi

| Pattern nei Residui | Problema | Soluzione |
|---------------------|----------|-----------|
| Trend (crescente/decrescente) | Manca una feature | Aggiungi variabili |
| Forma a U o arco | Relazione non lineare | Trasformazioni, polinomi |
| Cono (varianza crescente) | Eteroschedasticità | Log-trasformazione di y |
| Cluster | Gruppi diversi | Modelli separati, dummy |

In [ ]:
# ============================================
# DEMO: Analisi dei Residui
# ============================================

from sklearn.linear_model import LinearRegression
from scipy import stats

# Creiamo un dataset con relazione lineare + rumore
np.random.seed(42)
X_simple = np.linspace(0, 10, 100).reshape(-1, 1)
y_simple = 3 * X_simple.flatten() + 5 + np.random.normal(0, 2, 100)

# Fit modello
model_simple = LinearRegression()
model_simple.fit(X_simple, y_simple)
y_pred_simple = model_simple.predict(X_simple)
residui = y_simple - y_pred_simple

print("=" * 60)
print("📊 ANALISI DEI RESIDUI")
print("=" * 60)

print(f"\n📌 Statistiche Residui:")
print(f"   Media: {residui.mean():.4f} (ideale ≈ 0)")
print(f"   Std: {residui.std():.4f}")
print(f"   Min: {residui.min():.2f}, Max: {residui.max():.2f}")

# Visualizzazione completa
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residui vs Predetti
ax1 = axes[0, 0]
ax1.scatter(y_pred_simple, residui, alpha=0.6, s=50, c='steelblue', edgecolors='black')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('Valori Predetti', fontsize=12)
ax1.set_ylabel('Residui', fontsize=12)
ax1.set_title('📊 Residui vs Predetti (deve essere casuale!)', fontsize=14)
ax1.grid(True, alpha=0.3)

# 2. Q-Q Plot
ax2 = axes[0, 1]
stats.probplot(residui, dist="norm", plot=ax2)
ax2.set_title('📈 Q-Q Plot (deve seguire la linea)', fontsize=14)
ax2.grid(True, alpha=0.3)

# 3. Istogramma residui
ax3 = axes[1, 0]
ax3.hist(residui, bins=20, color='steelblue', edgecolor='black', alpha=0.7, density=True)
# Overlay normale
x_norm = np.linspace(residui.min(), residui.max(), 100)
ax3.plot(x_norm, stats.norm.pdf(x_norm, residui.mean(), residui.std()), 
         'r-', linewidth=2, label='Normale teorica')
ax3.set_xlabel('Residui', fontsize=12)
ax3.set_ylabel('Densità', fontsize=12)
ax3.set_title('📊 Distribuzione Residui (deve essere normale)', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Residui vs Ordine (sequenza)
ax4 = axes[1, 1]
ax4.plot(residui, 'o-', alpha=0.6, markersize=5, color='steelblue')
ax4.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax4.set_xlabel('Ordine Osservazione', fontsize=12)
ax4.set_ylabel('Residui', fontsize=12)
ax4.set_title('📈 Residui nel Tempo (no pattern!)', fontsize=14)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Test normalità
_, p_value = stats.shapiro(residui)
print(f"\n📈 Test di Shapiro-Wilk (normalità):")
print(f"   p-value = {p_value:.4f}")
print(f"   {'✅ Residui normali (p > 0.05)' if p_value > 0.05 else '❌ Residui NON normali'}")

---

## 6️⃣ Quando Usare Quale Metrica

### Flowchart Decisionale

```
Quale metrica uso?
        │
        ├── Vuoi comunicare al business? → MAPE (%)
        │
        ├── Vuoi penalizzare errori grandi? → RMSE
        │
        ├── Vuoi robustezza agli outlier? → MAE
        │
        ├── Vuoi confrontare modelli su scale diverse? → R²
        │
        └── Vuoi diagnosticare il modello? → Analisi Residui
```

### Tabella Riassuntiva

| Metrica | Unità | Outlier | Uso Principale |
|---------|-------|---------|----------------|
| **MAE** | Come y | Robusto | Errore tipico |
| **MSE** | y² | Sensibile | Ottimizzazione |
| **RMSE** | Come y | Sensibile | Errore "tipico" pesato |
| **MAPE** | % | Moderato | Comunicazione business |
| **R²** | Adimensionale | Moderato | Bontà complessiva |

### Regole Pratiche

1. **Report al business** → Usa MAPE ("Sbagliamo del 5%")
2. **Confronto modelli** → Usa RMSE o R²
3. **Dati con outlier** → Preferisci MAE
4. **Ottimizzazione** → MSE (derivabile)
5. **Sempre** → Guarda i residui!

---

## 🧠 Schema Mentale — Metriche di Regressione

```
                    ┌─────────────────────────────────────┐
                    │    METRICHE DI REGRESSIONE         │
                    │    "Quanto sbaglio?"               │
                    └──────────────┬──────────────────────┘
                                   │
         ┌─────────────────────────┼─────────────────────────┐
         ↓                         ↓                         ↓
┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐
│ ERRORE ASSOLUTO │      │ ERRORE RELATIVO │      │ BONTÀ MODELLO   │
│   MAE, RMSE     │      │     MAPE        │      │      R²         │
└────────┬────────┘      └────────┬────────┘      └────────┬────────┘
         │                        │                        │
    ┌────┴────┐              ┌────┴────┐              ┌────┴────┐
    │ MAE:    │              │ MAPE:   │              │ R²:     │
    │ Robusto │              │ Business│              │ 0-1     │
    │ k€, kg  │              │ talking │              │ Varianza│
    └─────────┘              └─────────┘              └─────────┘
         │
    ┌────┴────┐
    │ RMSE:   │
    │ Penaliz-│
    │ za gross│
    │ errori  │
    └─────────┘
```

---

## 🎯 Esercizi Svolti

### Esercizio 18.1 — Report Completo su Dataset Boston-like

**Obiettivo:** Valutare un modello di regressione sui prezzi delle case.

**Consegne:**
1. Genera un dataset di prezzi immobiliari
2. Addestra una regressione lineare
3. Calcola tutte le metriche: MAE, RMSE, MAPE, R²
4. Analizza i residui
5. Produci un report testuale interpretabile

In [ ]:
# ============================================
# ESERCIZIO 18.1 — SOLUZIONE COMPLETA
# Report Completo Modello di Regressione
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("=" * 60)
print("ESERCIZIO 18.1 — Report Completo Modello di Regressione")
print("=" * 60)

# 1. Generazione dataset immobiliare
n_case = 500

# Feature
mq = np.random.normal(100, 30, n_case)  # Metri quadri
stanze = np.random.randint(2, 6, n_case)  # Numero stanze
eta = np.random.randint(0, 50, n_case)  # Età edificio
distanza_centro = np.random.exponential(3, n_case)  # Km dal centro

# Prezzo (target) = combinazione lineare + rumore
prezzo = (
    50000 +                    # Base
    2000 * mq +               # €2000/mq
    15000 * stanze +          # €15000/stanza
    -500 * eta +              # -€500/anno di età
    -8000 * distanza_centro + # -€8000/km dal centro
    np.random.normal(0, 30000, n_case)  # Rumore
)
prezzo = np.clip(prezzo, 50000, None)  # Minimo 50k

df = pd.DataFrame({
    'mq': mq,
    'stanze': stanze,
    'eta_edificio': eta,
    'km_centro': distanza_centro,
    'prezzo': prezzo
})

print("\n📊 Dataset:")
print(df.describe().round(2))

# 2. Split e addestramento
X = df[['mq', 'stanze', 'eta_edificio', 'km_centro']]
y = df['prezzo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 3. Calcolo metriche
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
r2 = r2_score(y_test, y_pred)

# Adjusted R²
n = len(y_test)
p = X_test.shape[1]
r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Residui
residui = y_test - y_pred

print("\n" + "=" * 60)
print("📊 REPORT VALUTAZIONE MODELLO")
print("=" * 60)

print(f"""
┌────────────────────────────────────────────────────────────┐
│                    METRICHE DI ERRORE                      │
├────────────────────────────────────────────────────────────┤
│  MAE (Mean Absolute Error)     │  €{mae:>15,.0f}          │
│  RMSE (Root Mean Squared Error)│  €{rmse:>15,.0f}          │
│  MAPE (Mean Absolute % Error)  │   {mape:>14.2f}%          │
├────────────────────────────────────────────────────────────┤
│                    BONTÀ DEL MODELLO                       │
├────────────────────────────────────────────────────────────┤
│  R² (Coefficiente Determinazione)  │     {r2:>10.4f}       │
│  R² Adjusted                       │     {r2_adj:>10.4f}       │
└────────────────────────────────────────────────────────────┘
""")

# 4. Analisi residui
print("📈 ANALISI RESIDUI:")
print(f"   Media residui: €{residui.mean():,.0f} (ideale ≈ 0)")
print(f"   Std residui: €{residui.std():,.0f}")

_, shapiro_p = stats.shapiro(residui.sample(min(50, len(residui)), random_state=42))
print(f"   Test normalità (Shapiro): p = {shapiro_p:.4f}")

# 5. Visualizzazione
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Predetti vs Reali
ax1 = axes[0, 0]
ax1.scatter(y_test, y_pred, alpha=0.5, s=30, c='steelblue', edgecolors='black')
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
ax1.set_xlabel('Prezzo Reale (€)', fontsize=12)
ax1.set_ylabel('Prezzo Predetto (€)', fontsize=12)
ax1.set_title(f'🎯 Predizioni vs Reali (R² = {r2:.3f})', fontsize=14)
ax1.grid(True, alpha=0.3)

# Residui vs Predetti
ax2 = axes[0, 1]
ax2.scatter(y_pred, residui, alpha=0.5, s=30, c='steelblue', edgecolors='black')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Prezzo Predetto (€)', fontsize=12)
ax2.set_ylabel('Residui (€)', fontsize=12)
ax2.set_title('📊 Residui vs Predetti', fontsize=14)
ax2.grid(True, alpha=0.3)

# Distribuzione residui
ax3 = axes[1, 0]
ax3.hist(residui, bins=30, color='steelblue', edgecolor='black', alpha=0.7, density=True)
x_norm = np.linspace(residui.min(), residui.max(), 100)
ax3.plot(x_norm, stats.norm.pdf(x_norm, residui.mean(), residui.std()), 'r-', linewidth=2)
ax3.set_xlabel('Residui (€)', fontsize=12)
ax3.set_ylabel('Densità', fontsize=12)
ax3.set_title('📊 Distribuzione Residui', fontsize=14)
ax3.grid(True, alpha=0.3)

# Importanza coefficienti
ax4 = axes[1, 1]
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficiente': model.coef_
}).sort_values('Coefficiente', key=abs, ascending=True)

colors = ['green' if c > 0 else 'red' for c in coef_df['Coefficiente']]
ax4.barh(coef_df['Feature'], coef_df['Coefficiente'], color=colors, edgecolor='black')
ax4.axvline(x=0, color='black', linewidth=1)
ax4.set_xlabel('Coefficiente (€)', fontsize=12)
ax4.set_title('📊 Coefficienti del Modello', fontsize=14)
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# 6. Report business
print("\n" + "=" * 60)
print("📋 REPORT PER IL BUSINESS")
print("=" * 60)
print(f"""
📌 SINTESI:
   Il modello predice i prezzi con un errore medio del {mape:.1f}%.
   In termini assoluti, l'errore tipico è di €{mae:,.0f}.
   
📈 INTERPRETAZIONE COEFFICIENTI:
   • Ogni mq in più: +€{model.coef_[0]:,.0f}
   • Ogni stanza in più: +€{model.coef_[1]:,.0f}
   • Ogni anno di età: €{model.coef_[2]:,.0f}
   • Ogni km dal centro: €{model.coef_[3]:,.0f}
   
✅ QUALITÀ: R² = {r2:.2%} — il modello spiega {r2:.0%} della variabilità dei prezzi.
""")

---

### Esercizio 18.2 — Confronto Modelli con Diverse Metriche

**Obiettivo:** Confrontare 3 modelli di regressione usando metriche diverse.

**Consegne:**
1. Addestra Linear Regression, Ridge, Random Forest
2. Calcola MAE, RMSE, MAPE, R² per ciascuno
3. Visualizza il confronto
4. Quale modello sceglieresti? Dipende dalla metrica!

In [ ]:
# ============================================
# ESERCIZIO 18.2 — SOLUZIONE COMPLETA
# Confronto Modelli con Metriche Diverse
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("=" * 60)
print("ESERCIZIO 18.2 — Confronto Modelli di Regressione")
print("=" * 60)

# Dataset (riuso quello dell'esercizio precedente)
n_case = 500
mq = np.random.normal(100, 30, n_case)
stanze = np.random.randint(2, 6, n_case)
eta = np.random.randint(0, 50, n_case)
distanza_centro = np.random.exponential(3, n_case)

prezzo = (50000 + 2000 * mq + 15000 * stanze - 500 * eta - 8000 * distanza_centro +
          np.random.normal(0, 30000, n_case))
prezzo = np.clip(prezzo, 50000, None)

X = np.column_stack([mq, stanze, eta, distanza_centro])
y = prezzo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Definizione modelli
modelli = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1)': Ridge(alpha=1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

# 2. Training e valutazione
risultati = []

for nome, modello in modelli.items():
    modello.fit(X_train, y_train)
    y_pred = modello.predict(X_test)
    
    risultati.append({
        'Modello': nome,
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAPE': mean_absolute_percentage_error(y_test, y_pred) * 100,
        'R²': r2_score(y_test, y_pred)
    })

df_risultati = pd.DataFrame(risultati)

print("\n📊 RISULTATI:")
print(df_risultati.to_string(index=False))

# 3. Visualizzazione
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metriche = ['MAE', 'RMSE', 'MAPE', 'R²']
titoli = ['📊 MAE (più basso = meglio)', '📊 RMSE (più basso = meglio)', 
          '📊 MAPE % (più basso = meglio)', '📊 R² (più alto = meglio)']
colori = ['#3498db', '#e74c3c', '#27ae60']

for idx, (metrica, titolo) in enumerate(zip(metriche, titoli)):
    ax = axes[idx // 2, idx % 2]
    valori = df_risultati[metrica].values
    
    # Evidenzia il migliore
    if metrica == 'R²':
        best_idx = np.argmax(valori)
    else:
        best_idx = np.argmin(valori)
    
    colors = ['green' if i == best_idx else colori[i] for i in range(len(valori))]
    
    bars = ax.bar(df_risultati['Modello'], valori, color=colors, edgecolor='black')
    ax.set_ylabel(metrica, fontsize=12)
    ax.set_title(titolo, fontsize=14)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Etichette
    for bar, val in zip(bars, valori):
        formato = f'{val:.2f}' if metrica in ['MAPE', 'R²'] else f'{val:,.0f}'
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02 * max(valori),
                formato, ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# 4. Analisi vincitori
print("\n" + "=" * 60)
print("🏆 ANALISI VINCITORI")
print("=" * 60)

vincitori = {
    'MAE': df_risultati.loc[df_risultati['MAE'].idxmin(), 'Modello'],
    'RMSE': df_risultati.loc[df_risultati['RMSE'].idxmin(), 'Modello'],
    'MAPE': df_risultati.loc[df_risultati['MAPE'].idxmin(), 'Modello'],
    'R²': df_risultati.loc[df_risultati['R²'].idxmax(), 'Modello']
}

for metrica, vincitore in vincitori.items():
    print(f"   {metrica}: {vincitore}")

# Conta vittorie
from collections import Counter
conteggio = Counter(vincitori.values())
vincitore_assoluto = conteggio.most_common(1)[0]

print(f"\n🏅 Vincitore assoluto: {vincitore_assoluto[0]} ({vincitore_assoluto[1]}/4 metriche)")

print("\n💡 Nota:")
print("   Random Forest spesso vince perché cattura relazioni non lineari.")
print("   Ma è più lento e meno interpretabile della regressione lineare.")

---

### Esercizio 18.3 — Diagnosi Modello con Residui Problematici

**Obiettivo:** Identificare problemi in un modello attraverso l'analisi dei residui.

**Consegne:**
1. Crea un dataset con relazione NON lineare
2. Fitta un modello lineare (sbagliato!)
3. Analizza i residui e identifica il problema
4. Correggi il modello e confronta

In [ ]:
# ============================================
# ESERCIZIO 18.3 — SOLUZIONE COMPLETA
# Diagnosi con Analisi Residui
# ============================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("=" * 60)
print("ESERCIZIO 18.3 — Diagnosi con Analisi Residui")
print("=" * 60)

# 1. Creiamo dati con relazione QUADRATICA
X = np.linspace(0, 10, 200).reshape(-1, 1)
y_true_curve = 5 + 2*X.flatten() + 0.5*X.flatten()**2  # Relazione quadratica!
y = y_true_curve + np.random.normal(0, 2, 200)

print("\n📌 Dataset: relazione QUADRATICA nascosta")
print("   y = 5 + 2x + 0.5x² + rumore")

# 2. Fittiamo un modello LINEARE (sbagliato!)
model_lineare = LinearRegression()
model_lineare.fit(X, y)
y_pred_lin = model_lineare.predict(X)
residui_lin = y - y_pred_lin

r2_lin = r2_score(y, y_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(y, y_pred_lin))

print(f"\n📊 Modello Lineare:")
print(f"   R² = {r2_lin:.4f}")
print(f"   RMSE = {rmse_lin:.4f}")

# 3. Visualizzazione problema
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Fit lineare
ax1 = axes[0, 0]
ax1.scatter(X, y, alpha=0.5, s=20, label='Dati')
ax1.plot(X, y_pred_lin, 'r-', linewidth=2, label='Lineare')
ax1.set_xlabel('X')
ax1.set_ylabel('y')
ax1.set_title('❌ Modello Lineare (sbagliato!)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Residui vs X
ax2 = axes[0, 1]
ax2.scatter(X, residui_lin, alpha=0.5, s=20, c='red')
ax2.axhline(y=0, color='black', linestyle='--')
ax2.set_xlabel('X')
ax2.set_ylabel('Residui')
ax2.set_title('⚠️ Residui: PATTERN A U!', fontsize=12)
ax2.grid(True, alpha=0.3)

# Residui vs Predetti
ax3 = axes[0, 2]
ax3.scatter(y_pred_lin, residui_lin, alpha=0.5, s=20, c='red')
ax3.axhline(y=0, color='black', linestyle='--')
ax3.set_xlabel('Predetti')
ax3.set_ylabel('Residui')
ax3.set_title('⚠️ Non casuali = problema!', fontsize=12)
ax3.grid(True, alpha=0.3)

# 4. Correggiamo: modello POLINOMIALE
model_poly = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('linear', LinearRegression())
])
model_poly.fit(X, y)
y_pred_poly = model_poly.predict(X)
residui_poly = y - y_pred_poly

r2_poly = r2_score(y, y_pred_poly)
rmse_poly = np.sqrt(mean_squared_error(y, y_pred_poly))

print(f"\n📊 Modello Polinomiale (grado 2):")
print(f"   R² = {r2_poly:.4f}")
print(f"   RMSE = {rmse_poly:.4f}")

# Fit polinomiale
ax4 = axes[1, 0]
ax4.scatter(X, y, alpha=0.5, s=20, label='Dati')
ax4.plot(X, y_pred_poly, 'g-', linewidth=2, label='Polinomiale')
ax4.set_xlabel('X')
ax4.set_ylabel('y')
ax4.set_title('✅ Modello Polinomiale (corretto!)', fontsize=12)
ax4.legend()
ax4.grid(True, alpha=0.3)

# Residui corretti vs X
ax5 = axes[1, 1]
ax5.scatter(X, residui_poly, alpha=0.5, s=20, c='green')
ax5.axhline(y=0, color='black', linestyle='--')
ax5.set_xlabel('X')
ax5.set_ylabel('Residui')
ax5.set_title('✅ Residui: casuali!', fontsize=12)
ax5.grid(True, alpha=0.3)

# Confronto metriche
ax6 = axes[1, 2]
modelli = ['Lineare', 'Polinomiale']
r2_vals = [r2_lin, r2_poly]
colors = ['red', 'green']

bars = ax6.bar(modelli, r2_vals, color=colors, edgecolor='black')
ax6.set_ylabel('R²')
ax6.set_title('📊 Confronto R²', fontsize=12)
ax6.set_ylim(0, 1)

for bar, val in zip(bars, r2_vals):
    ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{val:.3f}', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# 5. Conclusione
print("\n" + "=" * 60)
print("📋 DIAGNOSI COMPLETATA")
print("=" * 60)
print(f"""
❌ PROBLEMA IDENTIFICATO:
   I residui del modello lineare mostravano un pattern a U.
   Questo indica una relazione NON LINEARE nei dati.

✅ SOLUZIONE:
   Usando un modello polinomiale (grado 2), i residui diventano
   casuali e il R² passa da {r2_lin:.3f} a {r2_poly:.3f}.

💡 LEZIONE:
   SEMPRE analizzare i residui! Le metriche da sole non bastano.
   Un R² discreto ({r2_lin:.2f}) può nascondere un modello sbagliato.
""")

---

## 📌 Conclusione e Bignami

### ✅ Cosa Abbiamo Imparato

1. **MAE** — errore medio assoluto, robusto agli outlier
2. **RMSE** — penalizza errori grandi, stessa unità di y
3. **MAPE** — errore percentuale, perfetto per il business
4. **R²** — varianza spiegata, occhio all'interpretazione!
5. **Residui** — SEMPRE analizzarli per diagnosticare il modello

---

## 📋 BIGNAMI — Metriche di Regressione

| Metrica | Formula | Unità | Outlier | Uso |
|---------|---------|-------|---------|-----|
| **MAE** | Σ\|y-ŷ\|/n | Come y | Robusto | Errore tipico |
| **MSE** | Σ(y-ŷ)²/n | y² | Sensibile | Ottimizzazione |
| **RMSE** | √MSE | Come y | Sensibile | Errore penalizzato |
| **MAPE** | Σ\|y-ŷ\|/y/n | % | Moderato | Report business |
| **R²** | 1-SS_res/SS_tot | [-∞,1] | Moderato | Bontà modello |

---

### 🎯 Checklist Valutazione Modello

```
□ Calcola MAE, RMSE, R² (minimo)
□ Se per business → aggiungi MAPE
□ Plotta predetti vs reali (deve essere diagonale)
□ Plotta residui vs predetti (deve essere casuale!)
□ Controlla normalità residui (Q-Q plot)
□ Confronta con baseline (media, modello semplice)
```

---

### 🔧 Codice Rapido di Riferimento

```python
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error, 
    mean_absolute_percentage_error,
    r2_score
)

# Tutte le metriche in un colpo
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = mean_absolute_percentage_error(y_true, y_pred) * 100
r2 = r2_score(y_true, y_pred)

# Adjusted R²
n, p = len(y), X.shape[1]
r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Residui
residui = y_true - y_pred
```

---

### ⚠️ Errori Comuni da Evitare

| ❌ Errore | ✅ Soluzione |
|-----------|-------------|
| Guardare solo R² | Analizza anche residui |
| Ignorare MAPE | Usalo per comunicare |
| Non plottare residui | Pattern = modello sbagliato |
| R² alto = modello perfetto | Può essere overfitting! |
| Usare MSE per interpretare | Usa RMSE o MAE |

---

### 🚀 Prossimi Passi
Nella **Lezione 19** vedremo il **Clustering: K-Means e DBSCAN** — algoritmi non supervisionati per scoprire gruppi nei dati!